In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
from collections import Counter

In [2]:
def case_control_func1(tbv, tick):
    #files
    case_county=pd.read_excel(r'../1_clean/output/tbv_county_distribution.xlsx')
    case_county['id']=case_county['id'].astype(str)
    case_county['country']=case_county['country'].astype(str)
    case_county['province']=case_county['province'].astype(str)
    case_county['county']=case_county['county'].astype(str) 
    case_county=case_county[case_county[tbv]>=1].reset_index(drop=True)
    case_county=case_county[['id', 'country', 'province', 'county']]
    case_county['status']=[1]*case_county.shape[0]

    tick_surveyed=pd.read_excel(r'../1_clean/output/tick_county_distribution.xlsx')
    tick_surveyed['id']=tick_surveyed['id'].astype(str)
    tick_surveyed['country']=tick_surveyed['country'].astype(str)
    tick_surveyed['province']=tick_surveyed['province'].astype(str)
    tick_surveyed['county']=tick_surveyed['county'].astype(str) 
    tick_surveyed=tick_surveyed[tick_surveyed[tick]==0].reset_index(drop=True)

    high_risk_county=pd.read_excel(r'output/tick/hrc_%s.xlsx'%tick)
    high_risk_county['id']=high_risk_county['id'].astype(str)
    high_risk_county['country']=high_risk_county['country'].astype(str)
    high_risk_county['province']=high_risk_county['province'].astype(str)
    high_risk_county['county']=high_risk_county['county'].astype(str)

    col_id=[]
    col_country=[]
    col_province=[]
    col_county=[]
    for i in range(tick_surveyed.shape[0]):
        id=tick_surveyed['id'][i]
        country=tick_surveyed['country'][i]
        province=tick_surveyed['province'][i]
        county=tick_surveyed['county'][i]
        tmp=case_county[(case_county['id']==id) & (case_county['country']==country) & (case_county['province']==province) & (case_county['county']==county)].reset_index(drop=True)
        if tmp.shape[0]==0: #该县区没有报告蜱虫疾病病例
            tmp1=high_risk_county[(high_risk_county['id']==id) & (high_risk_county['country']==country) & (high_risk_county['province']==province) & (high_risk_county['county']==county)].reset_index(drop=True)
            if tmp1.shape[0]==0: #该县区不是该蜱虫的高风险区域
                col_id.append(id)
                col_country.append(country)
                col_province.append(province)
                col_county.append(county)
    control_county=pd.DataFrame({'id': col_id, 'country': col_country, 'province': col_province, 'county': col_county})
    control_county['status']=[0]*control_county.shape[0]                    

    #check county in control county not in case county
    for i in range(control_county.shape[0]):
        id=control_county['id'][i]
        country=control_county['country'][i]
        province=control_county['province'][i]
        county=control_county['county'][i]
        tmp=case_county[(case_county['id']==id) & (case_county['country']==country) & (case_county['province']==province) & (case_county['county']==county)].reset_index(drop=True)
        if tmp.shape[0]!=0:
            print('error, county both in control and case county', id, country, province, county)
    
    combine=pd.concat([case_county, control_county], ignore_index=True)
    combine.to_excel(r'output/tbv/case_control_%s.xlsx'%tbv, index=False)
    print(tbv, case_county.shape[0], control_county.shape[0])

In [3]:
tbv=['Alongshan Virus', 'Dabie Bandavirus', 'Tick-borne Encephalitis Virus', 'Wetland Virus']
tick=['Ixodes persulcatus', 'Haemaphysalis longicornis', 'Ixodes persulcatus', 'Haemaphysalis concinna']

for i in range(len(tick)):
    case_control_func1(tbv[i], tick[i])

Alongshan Virus 18 165
Dabie Bandavirus 251 137
Tick-borne Encephalitis Virus 212 150
Wetland Virus 10 240


In [4]:
def case_control_func2(tbv):
    #files
    case_county=pd.read_excel(r'../1_clean/output/tbv_county_distribution.xlsx')
    case_county['id']=case_county['id'].astype(str)
    case_county['country']=case_county['country'].astype(str)
    case_county['province']=case_county['province'].astype(str)
    case_county['county']=case_county['county'].astype(str) 
    case_county=case_county[case_county[tbv]>=1].reset_index(drop=True)
    case_county=case_county[['id', 'country', 'province', 'county']]
    case_county['status']=[1]*case_county.shape[0]

    tick_surveyed=pd.read_excel(r'../1_clean/output/tick_county_distribution.xlsx')
    tick_surveyed['id']=tick_surveyed['id'].astype(str)
    tick_surveyed['country']=tick_surveyed['country'].astype(str)
    tick_surveyed['province']=tick_surveyed['province'].astype(str)
    tick_surveyed['county']=tick_surveyed['county'].astype(str) 
    
    # print(tick_surveyed.shape[0])

    if tbv=='Beiji Nairovirus':
        ticks=['Ixodes persulcatus', 'Ixodes crenulatus', 'Dermacentor silvarum']
    elif tbv=='Songling Virus':
        ticks=['Ixodes crenulatus', 'Haemaphysalis longicornis']
    elif tbv=='Yezo Virus':
        ticks=['Ixodes persulcatus', 'Haemaphysalis megaspinosa', 'Ixodes ovatus']

    control_county=dict()
    for i in range(len(ticks)):
        tick_surveyed=tick_surveyed[tick_surveyed[ticks[i]]==0].reset_index(drop=True)
        high_risk_county=pd.read_excel(r'output/tick/hrc_%s.xlsx'%ticks[i])
        high_risk_county['id']=high_risk_county['id'].astype(str)
        high_risk_county['country']=high_risk_county['country'].astype(str)
        high_risk_county['province']=high_risk_county['province'].astype(str)
        high_risk_county['county']=high_risk_county['county'].astype(str)
        col_id=[]
        col_country=[]
        col_province=[]
        col_county=[]
        for j in range(tick_surveyed.shape[0]):
            id=tick_surveyed['id'][j]
            country=tick_surveyed['country'][j]
            province=tick_surveyed['province'][j]
            county=tick_surveyed['county'][j]
            tmp=case_county[(case_county['id']==id) & (case_county['country']==country) & (case_county['province']==province) & (case_county['county']==county)].reset_index(drop=True)
            if tmp.shape[0]==0: #该县区没有报告蜱虫疾病病例
                tmp1=high_risk_county[(high_risk_county['id']==id) & (high_risk_county['country']==country) & (high_risk_county['province']==province) & (high_risk_county['county']==county)].reset_index(drop=True)
                if tmp1.shape[0]==0: #该县区不是该蜱虫的高风险区域
                    col_id.append(id)
                    col_country.append(country)
                    col_province.append(province)
                    col_county.append(county)
        control_county[i]=pd.DataFrame({'id': col_id, 'country': col_country, 'province': col_province, 'county': col_county})

    control_county=pd.concat([control_county[i] for i in range(len(ticks))], ignore_index=True)
    control_county['id']=control_county['id'].astype(str)
    control_county['country']=control_county['country'].astype(str)
    control_county['province']=control_county['province'].astype(str)
    control_county['county']=control_county['county'].astype(str)    
    # control_county=control_county.drop_duplicates().reset_index(drop=True)
    control_county = control_county.groupby(['id', 'country', 'province', 'county']).size().reset_index(name='status')
    control_county=control_county[control_county['status']>=1].reset_index(drop=True)
    control_county=control_county[['id', 'country', 'province', 'county']]
    control_county['status']=[0]*control_county.shape[0]
    print(control_county.shape[0])
    
    #check county in control county not in case county
    for i in range(control_county.shape[0]):
        id=control_county['id'][i]
        country=control_county['country'][i]
        province=control_county['province'][i]
        county=control_county['county'][i]
        tmp=case_county[(case_county['id']==id) & (case_county['country']==country) & (case_county['province']==province) & (case_county['county']==county)].reset_index(drop=True)
        if tmp.shape[0]!=0:
            print('error, county both in control and case county', id, country, province, county)
    
    combine=pd.concat([case_county, control_county], ignore_index=True)
    combine.to_excel(r'output/tbv/case_control_%s.xlsx'%tbv, index=False)
    print(tbv, case_county.shape[0], control_county.shape[0])

In [5]:
tbv=['Beiji Nairovirus', 'Songling Virus', 'Yezo Virus']
for i in range(len(tbv)):
    case_control_func2(tbv[i])

209
Beiji Nairovirus 46 209
209
Songling Virus 11 209
232
Yezo Virus 27 232


In [6]:
def predict(tbv):
    risk_factor_data=pd.read_excel(r'output/risk_factor/tbv.xlsx')
    risk_factor_data['id']=risk_factor_data['id'].astype(str)
    risk_factor_data['country']=risk_factor_data['country'].astype(str)
    risk_factor_data['province']=risk_factor_data['province'].astype(str)
    risk_factor_data['county']=risk_factor_data['county'].astype(str)
    risk_factors=risk_factor_data.columns[4:]

    tick_data=pd.read_excel(r'../1_clean/output/tbv_county_distribution.xlsx')
    tick_data=tick_data[['id', 'country', 'province', 'county', tbv]]
    tick_data['id']=tick_data['id'].astype(str)
    tick_data['country']=tick_data['country'].astype(str)
    tick_data['province']=tick_data['province'].astype(str)
    tick_data['county']=tick_data['county'].astype(str)
    
    #combine and split
    complete_data=risk_factor_data.merge(tick_data, left_on=['id', 'country', 'province', 'county'], right_on=['id', 'country', 'province', 'county'], how='left')
    complete_data[tbv]=complete_data[tbv].fillna(-999)
    complete_data[tbv]=complete_data[tbv].apply(lambda x: 1 if x > 0 else x)
        
    survey_data=pd.read_excel(r'output/tbv/case_control_%s.xlsx'%tbv)
    survey_data['id']=survey_data['id'].astype(str)
    survey_data['country']=survey_data['country'].astype(str)
    survey_data['province']=survey_data['province'].astype(str)
    survey_data['county']=survey_data['county'].astype(str)  
    survey_data=survey_data.merge(risk_factor_data, left_on=['id', 'country', 'province', 'county'], right_on=['id', 'country', 'province', 'county'], how='left')
    
    
    #step 1, identify key risk factors
    rel_con=pd.DataFrame({'Key': risk_factors})   
    for i in range(10):
            train, test = train_test_split(survey_data, test_size=0.25,  random_state=i)
            while test[test['status']==1].shape[0]<1 or train[train['status']==1].shape[0]<1:
                train, test = train_test_split(survey_data, test_size=0.25)
            train_x = train[risk_factors]
            train_y = train['status']
            # train_weight =  train['rescaled']
            model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.005, max_depth=5, subsample=0.75, n_estimators=3000,  random_state=i) # colsample_bytree=1
            model.fit(train_x, train_y)
            importances = model.get_booster().get_score(importance_type='weight')
            importances_df=pd.DataFrame(list(importances.items()), columns=['Key', 'Value'])
            importances_df=importances_df.rename(columns={'Value': 'Value_%d'%i})
            rel_con=rel_con.merge(importances_df, left_on='Key', right_on='Key', how='left')
    rel_con['sum'] = rel_con.iloc[:, 1:].sum(axis=1)
    rel_con['rc']=rel_con['sum']/rel_con['sum'].sum()
    key_risk_df=rel_con[rel_con['rc']>=0.02].reset_index(drop=True)
    key_risk_factors=key_risk_df['Key'].values
    
    #step 2, predict use key risk factors
    rel_con=pd.DataFrame({'Key': key_risk_factors})
    preds=[]
    auc_value=[]
    high_risk=[]
    for i in range(100):
        train, test = train_test_split(survey_data, test_size=0.25,  random_state=i)
        while test[test['status']==1].shape[0]<1 or train[train['status']==1].shape[0]<1:
            train, test = train_test_split(survey_data, test_size=0.25)

        train_x = train[key_risk_factors]
        train_y = train['status']
        # train_weight =  train['rescaled']
        model = xgb.XGBClassifier(objective='binary:logistic', learning_rate=0.005, max_depth=5, subsample=0.75, n_estimators=3000,  random_state=i) # colsample_bytree=1
        model.fit(train_x, train_y)
        importances = model.get_booster().get_score(importance_type='weight')
        importances_df=pd.DataFrame(list(importances.items()), columns=['Key', 'Value'])
        importances_df=importances_df.rename(columns={'Value': 'Value_%d'%i})
        rel_con=rel_con.merge(importances_df, left_on='Key', right_on='Key', how='left')
        
        test_x = test[key_risk_factors]
        test_y = test['status']
        pred_test_y=model.predict_proba(test_x)[:,1]
        fpr, tpr, thresholds = roc_curve(test_y, pred_test_y)
        roc_auc = auc(fpr, tpr)
        auc_value.append(roc_auc)

        all_x=risk_factor_data[key_risk_factors]
        pred_all_y=model.predict_proba(all_x)[:,1]
        pred_df=pd.DataFrame({'id':risk_factor_data['id'], 'country': risk_factor_data['country'], 'province': risk_factor_data['province'], 'county':risk_factor_data['county'], 'round': [i]*risk_factor_data.shape[0], 'status':complete_data[tbv], 'pred':pred_all_y})
        preds.append(pred_df)

        high_risk_data=pred_df[pred_df['pred']>=0.5].reset_index(drop=True)
        high_risk.append(high_risk_data)

    #mean auc of testing set
    auc_mean=np.mean(auc_value)
    auc_ubd=np.percentile(auc_value, 97.5)
    auc_lbd=np.percentile(auc_value, 2.5)
    auc_df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd'], 'value': [auc_mean, auc_lbd, auc_ubd]})
    auc_df.to_excel(r'output/tbv/auc_%s.xlsx'%tbv, index=False)

    #relative contribution
    rel_con['row_sum']=rel_con.iloc[:, 1:].sum(axis=1)
    rel_con['std_sum']=rel_con['row_sum']/rel_con['row_sum'].sum()
    rel_con.to_excel(r'output/tbv/rc_%s.xlsx'%tbv, index=False)
    

    high_risk_df=pd.concat(high_risk, ignore_index=True)
    high_risk_df.to_excel(r'output/tbv/hrc_%s.xlsx'%tbv, index=False)

    #calcualte mean, std and 95% CI
    pred_df=pd.concat(preds, ignore_index=True)
    grouped = pred_df.groupby(['id', 'country', 'province', 'county', 'status'])['pred'].mean()
    grouped = grouped.reset_index()
    grouped.to_excel(r'output/tbv/predict_prob_%s.xlsx'%tbv, index=False)

    #high_risk_counties
    # high_risk_df=grouped[grouped['pred']>=0.5].reset_index(drop=True)
    # high_risk_df.to_excel(r'output/high_risk_county/tbv/%s.xlsx'%tbv, index=False)

In [3]:
tbvs=['Alongshan Virus',  'Beiji Nairovirus',  'Dabie Bandavirus', 'Songling Virus', 'Tick-borne Encephalitis Virus', 'Wetland Virus', 'Yezo Virus']
for tbv in tbvs:
    print(tbv)
    predict(tbv)

Alongshan Virus


NameError: name 'predict' is not defined

In [8]:
#AUC
df=pd.DataFrame({'index': ['mean', 'lbd', 'ubd']})
tbvs=['Alongshan Virus',  'Beiji Nairovirus',  'Dabie Bandavirus', 'Songling Virus', 'Tick-borne Encephalitis Virus', 'Wetland Virus', 'Yezo Virus']
for i in range(len(tbvs)):
    tmp=pd.read_excel(r'output/tbv/auc_%s.xlsx'%tbvs[i])
    df[tbvs[i]]=tmp['value']
dfn=df.T
dfn.to_excel(r'output/AUC_tbv.xlsx')

In [9]:
#relative contribution
risk_factor=pd.read_excel(r'output/risk_factor/tbv.xlsx')
df=pd.DataFrame({'factor': risk_factor.columns[4:]})
tbvs=['Alongshan Virus',  'Beiji Nairovirus',  'Dabie Bandavirus', 'Songling Virus', 'Tick-borne Encephalitis Virus', 'Wetland Virus', 'Yezo Virus']
for i in range(len(tbvs)):
    tmp=pd.read_excel(r'output/tbv/rc_%s.xlsx'%tbvs[i])
    tmp=tmp[['Key', 'std_sum']]
    tmp=tmp.rename(columns={'Key': 'factor', 'std_sum': tbvs[i]})
    df=df.merge(tmp, left_on='factor', right_on='factor', how='left')
df.to_excel(r'output/RC_tbv.xlsx')

In [4]:
tbvs=['Alongshan Virus',  'Beiji Nairovirus',  'Dabie Bandavirus', 'Songling Virus', 'Tick-borne Encephalitis Virus', 'Wetland Virus', 'Yezo Virus']
for i in range(len(tbvs)):
    #high_risk_county, scenario 1
    county_list=pd.read_excel(r'output/risk_factor/tbv.xlsx')
    county_list=county_list[['id', 'country', 'province', 'county']]
    county_list['id']=county_list['id'].astype(str)
    county_list['country']=county_list['country'].astype(str)
    county_list['province']=county_list['province'].astype(str)
    county_list['county']=county_list['county'].astype(str)

    tick_data=pd.read_excel(r'output/tbv/hrc_%s.xlsx'%tbvs[i])
    tick_data['id']=tick_data['id'].astype(str)
    tick_data['country']=tick_data['country'].astype(str)
    tick_data['province']=tick_data['province'].astype(str)
    tick_data['county']=tick_data['county'].astype(str)
    for j in range(100):
        tmp=tick_data[tick_data['round']==j].reset_index(drop=True)
        new_column='flag_%d'%j
        tmp[new_column]=[1]*tmp.shape[0]
        tmp=tmp[['id','country','province','county', new_column]]
        county_list=county_list.merge(tmp, left_on=['id','country','province','county'], right_on=['id','country','province','county'], how='left')
    county_list.fillna(0, inplace=True)
    county_list['status']=county_list.iloc[:, 4:].sum(axis=1)
    df=county_list[county_list['status']>=1].reset_index(drop=True)
    df=df[['id','country','province','county', 'status']]
    # df.rename(columns={'status': ticks[i]}, inplace=True)
    df.to_excel(r'output/tbv/hrc_%s_clean.xlsx'%tbvs[i], index=False)
    print(tbvs[i], df.shape[0])

Alongshan Virus 396
Beiji Nairovirus 634
Dabie Bandavirus 2216
Songling Virus 313
Tick-borne Encephalitis Virus 1844
Wetland Virus 259
Yezo Virus 432


In [5]:
obv_number=[]
obv_area=[]
obv_pop=[]
pred_number=[]
pred_area=[]
pred_pop=[]
col_tbv=[]
for tbv in tbvs:
    pred=pd.read_excel(r'output/tbv/hrc_%s_clean.xlsx'%tbv)
    pred['id']=pred['id'].astype(str)
    pred['country']=pred['country'].astype(str)
    pred['province']=pred['province'].astype(str)
    pred['county']=pred['county'].astype(str)

    obv=pd.read_excel(r'../1_clean/output/tbv_county_distribution.xlsx')
    obv=obv[['id', 'country', 'province', 'county', tbv]]
    obv=obv[obv[tbv]>=1].reset_index(drop=True)
    obv['id']=obv['id'].astype(str)
    obv['country']=obv['country'].astype(str)
    obv['province']=obv['province'].astype(str)
    obv['county']=obv['county'].astype(str)
    
    county_information=pd.read_excel(r'../2_riskfactor/output/population.xlsx')
    county_information['population']=county_information['area (km²)']*county_information['population density (/km²)']
    county_information['population']=county_information['population'].astype('int')
    county_information['id']=county_information['id'].astype(str)
    county_information['country']=county_information['country'].astype(str)
    county_information['province']=county_information['province'].astype(str)
    county_information['county']=county_information['county'].astype(str)

    number=0
    area=0
    population=0
    for j in range(pred.shape[0]):
        id=pred['id'][j]
        country=pred['country'][j]
        province=pred['province'][j]
        county=pred['county'][j]
        tmp=county_information[(county_information['id']==id) & (county_information['country']==country) & (county_information['province']==province) & (county_information['county']==county)].reset_index(drop=True)
        if tmp.shape[0]!=1:
            print('pred', tmp)
        else:
            number+=1
            area+=(tmp['area (km²)'][0])
            population+=(tmp['population'][0])
    pred_number.append(number)
    pred_area.append(area)
    pred_pop.append(population)

    number=0
    area=0
    population=0
    for j in range(obv.shape[0]):
        id=obv['id'][j]
        country=obv['country'][j]
        province=obv['province'][j]
        county=obv['county'][j]
        tmp=county_information[(county_information['id']==id) & (county_information['country']==country) & (county_information['province']==province) & (county_information['county']==county)].reset_index(drop=True)
        if tmp.shape[0]!=1:
            print('pred', tmp)
        else:
            number+=1
            area+=(tmp['area (km²)'][0])
            population+=(tmp['population'][0])
    obv_number.append(number)
    obv_area.append(area)
    obv_pop.append(population)   

    col_tbv.append(tbv)

col_1=[]
col_2=[]
col_3=[]
for i in range(len(col_tbv)):
    obv1=obv_number[i]
    pred1=pred_number[i]
    ratio1=100*(pred1-obv1)/obv1
    print(col_tbv[i], obv1, pred1, ratio1)
    col_1.append("%d/%d (%.1f)"%(pred1, obv1, ratio1))

    obv2=obv_area[i]/1e6
    pred2=pred_area[i]/1e6
    ratio2=100*(pred2-obv2)/obv2
    col_2.append("%.2f/%.2f (%.1f)"%(pred2, obv2, ratio2))

    obv3=obv_pop[i]/1e6
    pred3=pred_pop[i]/1e6
    ratio3=100*(pred3-obv3)/obv3
    col_3.append("%.2f/%.2f (%.1f)"%(pred3, obv3, ratio3))

dfn=pd.DataFrame({'tick': col_tbv, 'county': col_1, 'area': col_2, 'population': col_3})
dfn.to_excel(r'output/PC_tbv.xlsx', index=False) 

Alongshan Virus 18 396 2100.0
Beiji Nairovirus 46 634 1278.2608695652175
Dabie Bandavirus 251 2216 782.8685258964143
Songling Virus 11 313 2745.4545454545455
Tick-borne Encephalitis Virus 212 1844 769.811320754717
Wetland Virus 10 259 2490.0
Yezo Virus 27 432 1500.0
